In [77]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.layers import Dense,Dropout,GlobalAveragePooling2D,BatchNormalization 
from tensorflow.keras.applications import EfficientNetB0

In [78]:
import tensorflow as tf

# Paths
TRAIN_DIR = "../data/raw/NEU-DET/train"
VAL_DIR   = "../data/raw/NEU-DET/validation"

# Constants
IMG_SIZE = (224,224)
BATCH_SIZE = 32


In [79]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    VAL_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

Found 1440 files belonging to 6 classes.
Found 360 files belonging to 6 classes.


In [80]:
from tensorflow.keras.applications.efficientnet import preprocess_input

train_ds = train_ds.map(
    lambda x, y: (preprocess_input(tf.cast(x, tf.float32)), y)
)

val_ds = val_ds.map(
    lambda x, y: (preprocess_input(tf.cast(x, tf.float32)), y)
)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(AUTOTUNE)
val_ds   = val_ds.cache().prefetch(AUTOTUNE)



In [81]:
NUM_CLASSES = 6

In [82]:
base_model = EfficientNetB0(input_shape=IMG_SIZE + (3,), include_top=False, weights='imagenet')

In [83]:
base_model.trainable = False

In [84]:
data_augmentation = tf.keras.Sequential([
    
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.08),
    tf.keras.layers.RandomZoom(0.15),
    tf.keras.layers.RandomTranslation(0.08, 0.08),
])


In [85]:
inputs = tf.keras.Input(shape=IMG_SIZE + (3,))

# Data augmentation (training only)
x = data_augmentation(inputs)

# Feature extraction
x = base_model(x, training=False)

# Classification head
inputs = tf.keras.Input(shape=(224,224,3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(256, activation="relu")(x)
x = tf.keras.layers.Dropout(0.4)(x)
outputs = tf.keras.layers.Dense(6, activation="softmax")(x)

model = models.Model(inputs, outputs)

In [86]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [87]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        "models/defect_detector_efficientnet.keras",
        monitor="val_loss",
        save_best_only=True
    )
]


In [88]:
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
    callbacks=callbacks
)

Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 27s 453ms/step - accuracy: 0.9222 - loss: 0.2108 - val_accuracy: 0.8694 - val_loss: 0.4854
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 18s 399ms/step - accuracy: 0.9937 - loss: 0.0251 - val_accuracy: 0.9083 - val_loss: 0.3360
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 18s 395ms/step - accuracy: 0.9944 - loss: 0.0153 - val_accuracy: 0.9611 - val_loss: 0.1862
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 18s 403ms/step - accuracy: 0.9937 - loss: 0.0123 - val_accuracy: 0.9694 - val_loss: 0.1297
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 18s 406ms/step - accuracy: 0.9979 - loss: 0.0075 - val_accuracy: 0.9917 - val_loss: 0.0656
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 18s 395ms/step - accuracy: 0.9972 - loss: 0.0078 - val_accuracy: 0.9889 - val_loss: 0.0456
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 18s 403ms/step - accuracy: 0.9972 - loss: 0.0053 - val_accuracy: 0.9778 - val_loss: 0.0623
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 19s 414ms/step - accuracy: 0.9986 - loss: 0.0034 - val_accu

In [93]:
base_model.trainable = True

Fine_Tune_At = int(len(base_model.layers) * 0.8)

for layer in base_model.layers[:Fine_Tune_At]:
    layer.trainable = False

In [94]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [95]:
history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks
)

Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 39s 621ms/step - accuracy: 0.9660 - loss: 0.1379 - val_accuracy: 0.9944 - val_loss: 0.0124
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 23s 517ms/step - accuracy: 0.9799 - loss: 0.1022 - val_accuracy: 0.9889 - val_loss: 0.0368
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 23s 518ms/step - accuracy: 0.9924 - loss: 0.0284 - val_accuracy: 0.9694 - val_loss: 0.1905
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 23s 517ms/step - accuracy: 0.9924 - loss: 0.0312 - val_accuracy: 0.9667 - val_loss: 0.2616
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 23s 518ms/step - accuracy: 0.9875 - loss: 0.0794 - val_accuracy: 0.9806 - val_loss: 0.1256
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 23s 516ms/step - accuracy: 0.9944 - loss: 0.0193 - val_accuracy: 0.9944 - val_loss: 0.0147


In [96]:
model.save("models/defect_detector_finetuned.keras")
